In [ ]:
%pip install duckdb


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import pandas as pd
import numpy as np
import plotly
import ollama
import duckdb
import json
import re

In [25]:
Model_Name="llama3.2:latest"

In [7]:
con=duckdb.connect()
tablename="data"
file_path="food_delivery_data.csv"
con.execute(f"""
            CREATE TABLE {tablename} AS
            SELECT * FROM read_csv_auto('{file_path}')
            """)
df = con.execute(f"SELECT * FROM {tablename} WHERE NOT EXISTS (SELECT 1 FROM {tablename} WHERE COLUMNS(*) IS NULL)").fetchdf()

In [8]:
con.execute(f"PRAGMA table_info({tablename})").fetchdf()

,cid,name,type,notnull,dflt_value,pk
0,0,OrderID,BIGINT,False,None,False
1,1,CustomerID,BIGINT,False,None,False
2,2,OrderDate,DATE,False,None,False
3,3,DeliveryDistanceKM,DOUBLE,False,None,False
4,4,OrderAmount,VARCHAR,False,None,False
5,5,DiscountApplied,DOUBLE,False,None,False
6,6,DeliveryTimeMin,BIGINT,False,None,False


In [9]:
df.head()

,OrderID,CustomerID,OrderDate,DeliveryDistanceKM,OrderAmount,DiscountApplied,DeliveryTimeMin
0,8180,436,2024-11-27,17.37,$35.53,0.70,42
1,2138,265,2024-11-27,4.44,$36.58,9.36,52
2,8737,193,2024-10-31,19.66,$79.93,3.38,59
3,1913,769,2024-11-14,1.99,$23.16,1.44,11
4,1506,982,2025-01-05,7.26,$24.95,11.87,36


In [10]:
field_data = {col: df[col].sample(n=min(5, len(df))).tolist() for col in df.columns}

In [11]:
field_data

{'OrderID': [1539, 8588, 8033, 5203, 1684],
 'CustomerID': [223, 374, 919, 641, 167],
 'OrderDate': [Timestamp('2024-11-02 00:00:00'),
  Timestamp('2024-11-11 00:00:00'),
  Timestamp('2024-10-28 00:00:00'),
  Timestamp('2024-10-18 00:00:00'),
  Timestamp('2024-11-09 00:00:00')],
 'DeliveryDistanceKM': [6.53, 10.86, 14.37, 11.3, 7.53],
 'OrderAmount': ['$19.34', '$79.92', '$72.32', '$95.83', '$57.01'],
 'DiscountApplied': [8.07, 3.59, 13.52, 2.81, 14.87],
 'DeliveryTimeMin': [32, 30, 36, 37, 52]}

In [39]:
def extract_data_insight(prompt):   
    system_prompt = """
    You are a data analytics assistant. Your task is to extract structured information from user queries.
    Given a user's query, extract the following details:
    
    - **Service Type**: What kind of analytics the user is requesting (e.g., summary, correlation, prediction, visualization, etc.)
    - **Column Names**: Identify the relevant columns mentioned.
    - **Operation**: The type of operation the user wants (e.g., sum, average, group by, filter, etc.).
    
    **ALERT** other than the json file i dont need any other text or explaination or anthing your job is to just give me the json file 
    Respond in JSON format as follows:
    {
        "service": "<service_type>",
        "columns": ["col1", "col2", ...],
        "operation": "<operation_type>"
    }
    """
    
    response = ollama.chat(model="llama3.2:latest", messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ])
    print("Raw LLM Response:", response)
    content = response["message"].get("content", "")
    return content

In [43]:
user_query = "Scatter plot with 'DeliveryDistanceKM' on the x-axis and 'DeliveryTimeMin' on the y-axis"
result = extract_data_insight(user_query)
print(json.dumps(result, indent=2))

Raw LLM Response: model='llama3.2:latest' created_at='2025-03-02T14:35:18.0582774Z' done=True done_reason='stop' total_duration=2238132500 load_duration=32706200 prompt_eval_count=225 prompt_eval_duration=76000000 eval_count=31 eval_duration=2127000000 message=Message(role='assistant', content='{\n  "service": "visualization",\n  "columns": ["DeliveryDistanceKM", "DeliveryTimeMin"],\n  "operation": "none"\n}', images=None, tool_calls=None)
"{\n  \"service\": \"visualization\",\n  \"columns\": [\"DeliveryDistanceKM\", \"DeliveryTimeMin\"],\n  \"operation\": \"none\"\n}"


In [47]:
def generate_duckdb_code(content):
    prompt = f"""
    You are an AI assistant that generates Python code for performing operations on an 
    existing DuckDB DataFrame (`df`). Given a string in the form format of an dictionary describing a data operation, 
    generate a complete Python script that processes `df` accordingly.

    The  string in the format of an dictionary contains the following fields:
    - **service_type**: Specifies the type of operation to perform (e.g., "aggregation", "filtering", "sorting", "visualization").
    - **columns**: A list of columns involved in the operation.
    - **operation**: Describes the specific operation to perform (e.g., "sum", "average", "count", "filter where column X > Y", "scatter plot of column X vs column Y").

    Instructions:
    1. **Perform the operation** directly on the provided DataFrame `df` using DuckDB.
    2. **Return the complete Python script** that executes the operation.
    3. If the `service_type` is "visualization", generate the visualization using **Plotly**.

    Assume `df` is already available and represents a DuckDB DataFrame. The generated code should not reload the dataset.

    Generate the Python script that implements this operation on `df`:
    IMPORTANT:
    the output should only contain the requried code and nothing else
    """

    # Call the local LLM using Ollama
    response = ollama.chat(model="llama3.2:latest", messages=[{"role": "user", "content": prompt+content}])

    # Extract the response
    generated_code = response['message']['content']
    
    return generated_code

In [49]:
match = re.search(r'```python\n(.*?)\n```', generate_duckdb_code(result), re.DOTALL)

if match:
    extracted_code = match.group(1)
    
    # Execute the extracted code
    exec(extracted_code)
else:
    print("No valid Python code found.")

AttributeError: 'duckdb.duckdb.DuckDBPyConnection' object has no attribute 'is_empty'

In [50]:
import plotly.express as px
import pandas as pd

def perform_operation(df):
    # Get the specified columns
    columns = df[["DeliveryDistanceKM", "DeliveryTimeMin"]]

    if "service_type" == "visualization":
        # Generate scatter plot of DeliveryDistanceKM vs DeliveryTimeMin
        fig = px.scatter(columns, x="DeliveryDistanceKM", y="DeliveryTimeMin")
        
        return fig

df = pd.DataFrame({
    'DeliveryDistanceKM': [5.1, 6.4, 7.3],
    'DeliveryTimeMin': [17.8, 16.9, 18.0]
})

# Create a figure with the specified columns and operation
result = perform_operation(df)

if result is not None:
    # Show the plot
    import plotly.io as plotly_io
    plotly_io.show(result)
else:
    print("No visualization required")

No visualization required


In [51]:
df = con.execute("SELECT DeliveryDistanceKM, DeliveryTimeMin FROM data WHERE DeliveryDistanceKM IS NOT NULL AND DeliveryTimeMin IS NOT NULL").fetchdf()

# Generate a scatter plot using Plotly
fig = px.scatter(df, x="DeliveryDistanceKM", y="DeliveryTimeMin",
                 title="Delivery Distance vs Time",
                 labels={"DeliveryDistanceKM": "Distance (KM)", "DeliveryTimeMin": "Time (Minutes)"},
                 color_discrete_sequence=["blue"])

# Show the plot
fig.show()